In [1]:
# Copyright 2025 Narges Kurkani

# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
# 
#     http://www.apache.org/licenses/LICENSE-2.0
# 
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [2]:
# Note:
# This notebook is based on the original GenAI notebook from Google (Apache 2.0 License).
# I have added some description and changes.

# 2. RAG Using Chroma

Large Language models are great at generating text, but they don’t know everything — especially when it comes to real-time information or private data. Plus, they can only take in a limited amount of input at once.

To make them smarter and more relevant, we can combine them with a method called Retrieval-Augmented Generation (RAG). This lets the model “look things up” from your own documents before answering.


🧩 How RAG Works
Here’s the basic idea:

- Prepare your data by turning documents into vectors and storing them.

- Search the data to find the most relevant content when someone asks a question.

- Generate an answer using both the question and the retrieved information.

This way, your model can answer with up-to-date, domain-specific knowledge — even if it wasn’t part of the model’s original training.

the key steps in a Retrieval-Augmented Generation (RAG) system:

- Indexing: Convert documents into vector embeddings with associated metadata and store them in a vector database for quick searching.

- Retrieval: When a query is received, search the indexed database to find the most relevant documents based on similarity.

- Generation: Combine the query with the retrieved documents and use a language model to generate a tailored, context-aware answer.

In this project, We’ll create a document-based Q&A system using:

- Chroma – an open-source vector database that stores document embeddings and allows fast, similarity-based searches.

- A language model API (Gemini) – to generate final responses using the data retrieved from Chroma.

You'll embed your text data, store it with helpful metadata, and then query it based on user input to produce answers that are grounded in real context.

## 2.1. Install Required Packages

In [ ]:
!pip install -qU "google-genai==1.7.0" "chromadb==0.6.3"
!pip install chromadb
pip install PyPDF2

In [1]:
from google import genai
from google.genai import types

from IPython.display import Markdown

genai.__version__

'1.7.0'

## 2.2. Set up API key

You can grab API key from [AI Studio](https://aistudio.google.com/app/apikey). And then choose creat API key and save it in GOOGLE_API_KEY. 

In [ ]:
GOOGLE_API_KEY = 'API_Key'

client = genai.Client(api_key=GOOGLE_API_KEY)

## 2.3. Exploring Available Models

In this guide, we'll compute embeddings using the embedContent API method. To find a model that supports this method, we'll check the models.list endpoint and refer to the models page for more details about embedding models.

For this exercise, we'll use the latest generally available model, text-embedding-004, and we can also experiment with the experimental gemini-embedding-exp-03-07 model.

In [3]:
client = genai.Client(api_key=GOOGLE_API_KEY)

for m in client.models.list():
    if "embedContent" in m.supported_actions:
        print(m.name)

models/embedding-001
models/text-embedding-004
models/gemini-embedding-exp-03-07
models/gemini-embedding-exp


## 2.4. Data

Using PyPDF2 package, we can add pdf file in documents and ask question about it.

In [4]:
import PyPDF2

# Open and read the PDF file
pdf_path = 'usermanual.pdf'  # Replace with your PDF file path
with open(pdf_path, 'rb') as file:
    reader = PyPDF2.PdfReader(file)
    pdf_text = ""
    
    # Loop through all the pages and extract text
    for page in reader.pages:
        pdf_text += page.extract_text()

# Split the extracted text into documents (if you want to split based on sections or paragraphs)
documents = pdf_text.split('\n\n')  # Adjust splitting method as needed
print(documents)


["4IMPORTANT SAFEGUARDS\nWhen using electrical appliances, basic safety precautions should always be followed to reduce the risk\nof fire, electric shock, and/pr injury to persons including the following:1. Read all instructions carefully before using the coffee-mill.2 To protect against risk of fire, electrical shock and injury to persons, do not immerse cord, plug or\nappliance in water or other liquids.\n3. The appliance is not intended for use by persons (including children) with reduced physical, sensory\nor mental capacities, or lack of experience and knowledge, unless they have been given supervisionor instruction concerning use of the appliance by a person responsible for their safety. Childrenshould be supervised to ensure that they do not play with the appliance.\n4. Close supervision is necessary when  any appliance is used by or near children.5. Do not remove the lid until the blade has stopped completely. After releasing the push button the\nblade will come to a stop withi

## 2.5. Creating the embedding database with ChromaDB

We'll create a custom function to generate embeddings using the Gemini API for our retrieval system. In our setup, documents—stored in our database (ChromaDB)—are first embedded using the retrieval_document task type. Later, we’ll generate embeddings for search queries with the retrieval_query task type. 

In [5]:
from chromadb import Documents, EmbeddingFunction, Embeddings
from google.api_core import retry

from google.genai import types


# Define a helper to retry when per-minute quota is reached.
is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})


class GeminiEmbeddingFunction(EmbeddingFunction):
    # Specify whether to generate embeddings for documents, or queries
    document_mode = True

    @retry.Retry(predicate=is_retriable)
    def __call__(self, input: Documents) -> Embeddings:
        if self.document_mode:
            embedding_task = "retrieval_document"
        else:
            embedding_task = "retrieval_query"

        response = client.models.embed_content(
            model="models/text-embedding-004",
            contents=input,
            config=types.EmbedContentConfig(
                task_type=embedding_task,
            ),
        )
        return [e.values for e in response.embeddings]

Now create a Chroma database client that uses the `GeminiEmbeddingFunction` and populate the database with the documents you defined above.

In [6]:
import chromadb

DB_NAME = "googlecardb"

embed_fn = GeminiEmbeddingFunction()
embed_fn.document_mode = True

chroma_client = chromadb.Client()
db = chroma_client.get_or_create_collection(name=DB_NAME, embedding_function=embed_fn)

db.add(documents=documents, ids=[str(i) for i in range(len(documents))])

## 2.6. Retrieval: Find relevant documents

To search our Chroma database, we call the query method and switch our embedding generation to the retrieval_query mode.

In [ ]:
# Switch to query mode when generating embeddings.
embed_fn.document_mode = False

# Search the Chroma DB using the specified query.
query = "Appliences that repaied outside of factory are included warranty?"

result = db.query(query_texts=[query], n_results=1)
[all_passages] = result["documents"]

Markdown(all_passages[0])

4IMPORTANT SAFEGUARDS
When using electrical appliances, basic safety precautions should always be followed to reduce the risk
of fire, electric shock, and/pr injury to persons including the following:1. Read all instructions carefully before using the coffee-mill.2 To protect against risk of fire, electrical shock and injury to persons, do not immerse cord, plug or
appliance in water or other liquids.
3. The appliance is not intended for use by persons (including children) with reduced physical, sensory
or mental capacities, or lack of experience and knowledge, unless they have been given supervisionor instruction concerning use of the appliance by a person responsible for their safety. Childrenshould be supervised to ensure that they do not play with the appliance.
4. Close supervision is necessary when  any appliance is used by or near children.5. Do not remove the lid until the blade has stopped completely. After releasing the push button the
blade will come to a stop within 2 to 3 seconds.
6. Unplug from the outlet when not in use, before putting on or taking off parts, and before cleaning.
Allow to cool down before putting on or taking off parts, before cleaning the appliance, and beforestoring.
7. Do not operate any appliance with a damaged cord or plug or after the appliance malfunctions, or
has been dropped or damaged in any manner. Return the appliance to the nearest authorizedDe’Longhi service center for examination, repair or adjustment.
8. To reduce the risk of injury do not drape cord over the counter top or table top where it can be pul-
led on by children or tripped over unintentionally.
9. Do not let the cord hang over the edge of the table or counter, or touch hot surfaces. A short power
cord is provided to reduce risks resulting from becoming entangled in or tripping over a longer cord.Longer, detachable power supply cords or extension cords are available and may be used if care isexercised in their use. If an extension cord is used, the marked electrical rating should be at least asgreat as the electrical rating of the appliance.  The longer cord should be arranged so that it will notdrape over the counter-top or table top where it can be pulled on by children or tripped over.
10. Do not use outdoors.11. Do not place on or near a hot gas or electric burner, or in a heated oven.12. To disconnect, remove plug from wall outlet. Do not yank cord, instead grasp plug and pull to di-
sconnect.
13. Do not use the appliance for othe than intended use. Never chop ice or use the coffee mill other than
as described in the instructions.
14. Check grinding chamber foor presence of foreign obects before using.15. Blades are sharp. Handle them carefully.16. Make certain the voltage of the appliance is in accordance with your domestic supply.17. This appliance is for household use only. Any servicing other than cleaning and user maintenance
should be performed by the nearest authorized  De’Longhi Service Center.Do not immerse the appliance in water.To reduce the risk of fire or electric shock, do not dismantle the appliance. Repair should be doneby an authorized De’Longhi Service Center only.
18. The use of attachments not recommended or sold by the manufacturer may cause fire, electric shock
or injury.
19. Avoid contacting moving parts.20. This appliance has a polarized plug (one blade is wider than the other). To reduce the risk of electric
shock, this plug is intended to fit into a polarized outlet only one way. If the plug does not fit fully inthe outlet, reverse the plug. If it still does not fit, contact a qualified electrician. Do not attempt to mo-dify the plug in any way.
SAVE THESE INSTRUCTIONS5GB
FR
ES
ITDESCRIPTION OF THE
APPLIANCE
A.Cleaning brush
B.Coffee bean container’s cover
C.Coffee bean container
D.Cable winder
E.Cups number selection ring (KG49 only)
F.Indicator for degree of grinding (KG49 only)
PRELIMINARY CHECKS
Controls after transport
After removing the packaging, make sure the pro-
duct is complete and undamaged and that all ac-cessories are present.Do not use the appliance if it is visibly damaged.Contact the technical service center. 
Connecting the appliance
Important! Check that the power supply
voltage corresponds to the value indicated on therating plate on the bottom of the appliance. 
USING THE APPLIANCE
 Place the appliance on a flat surface. Unscrew the lid in a counter-clockwise di-
rection and lift it off (fig. 1).
 Fill the container with the desired amount of
coffee beans (fig. 2).
 Screw the lid in a clockwise direction, rota-
ting it until it can't be turned anymore (fig.3). A safety device will not allow operation ifthe lid is not inserted correctly. Do not attempt operation without the cover
being in place.
 Pull out the length of cord required and plug
into the socket (fig. 4).
 In select models (KG49), turn the cups num-
ber selection ring (E) to the number of cupsyou wish to grind (fig. 5). The amount of cof-fee will change depending on individual taste,the type and roasting of the coffee beans andthe degree of grinding selected: never fill overthe MAX level stamped on the coffee beancontainer.
 Press the cover to start the coffee mill (fig.
6) and keep it pressed until the right grindinglevel is obtained. 
Do not operate this appliance continuouslyfor more than 60 seconds. Allow applianceto cool down for 10 minutes before usingagain. 
 In models with indicator lights showing the
degree of grinding ( F) (coarse, medium,
fine), press the cover and then release itwhen the indicator lights up relative to the re-quired degree of grinding. Remember thatwhen the cover is released, even for a se-cond, the appliance resets and starts grin-ding again as if there were whole beansinside.
TIPS
 Always keep the lid of the coffee bean con-
tainer tightly closed to keep in all the flavorand aroma of the coffee beans.
 Only grind the amount of coffee required. Do not overfill the coffee bean container.
As an indication, calculate a tablespoon ofcoffee beans per cup (e.g. 4 spoonfuls of cof-fee beans for 4 cups of coffee). The idealamount depends on individual taste and willbe discovered through the use of the ma-chine.
 For models with indicators lights showing
the degree of grinding, remember that:COARSE grinding is ideal for percolators;MEDIUM is ideal for mocha coffeepots anddrip coffee makers;FINE is ideal for espresso coffee makers.
 To ensure that you always have the best qua-
lity in terms of freshness, always clean theappliance after each use, especially if coffeeis ground after grinding other products (e.g.spices). However it is inadvisable to grindoily beans (e.g. peanuts) or products that aretoo hard (e.g. rice).CLEANING
Danger of electric shock! Before perfor-
ming any cleaning operations, the machine mustbe turned off and disconnected from the poweroutlet. Never immerse the appliance in water.
 Unplug from the outlet.
 Wash the cover with water and mild soap.
Then thoroughly rinse and dry. 
 To clean easily, remove the coffee bean con-
tainer completely by turning it and unscre-wing it clockwise until it is released, then liftit off  (fig. 7).
 Remove the coffee grounds residues with the
provided brush (fig. 8). Never immerse thecoffee bean container in water to avoid da-mages.
 To replace the coffee bean container, rotate
counterclockwise until it locks in position.
 Clean the outside of the appliance with a soft
cloth. 
 When you store after use of the appliance,
wind the cable on the cable winder ( D).
67GB
FR
ES
ITThis warranty applies to all products with De’Longhi or Kenwood brand names.
LIMITED WARRANTY
What does the warranty cover?
We warrant each appliance to be free from defects in material and workmanship. Our obligation underthis warranty is limited to repair at our factory or authorized service center of any defective parts or partthereof, other than parts damaged in transit. In the event of a products replacement or return, the unitmust be returned transportation prepaid. The repaired or new model will be returned at the company’sexpense.This warranty shall apply only if the appliance is used in accordance with the factory directions, whichaccompany it, and on an Alternating current ( AC ) circuit.
How long does the coverage last?
This warranty runs for one year ( 1 ) from the purchase date found on your receipt and applies only tothe original purchaser for use.
What is not covered by the warranty?
The warranty does not cover defects or damage of the appliance, which result from repairs or alterationsto the appliance outside our factory or authorized service centers, nor shall it apply to any appliance,which has been subject to abuse, misuse, negligence or accidents.  Also, consequential and incidentaldamage resulting from the use of this product or arising out of any breach of contract or breach of thiswarranty are not recoverable under this warranty.  Some states do not allow the exclusion or limitationof incidental or consequential damage, so the above limitation may not apply to you.
How do you get service?
If repairs become necessary, see contact information below:
U.S. Residents:  Please contact our toll free hotline at 1-800-322-3848 or log onto our website at
www.delonghiusa.com. For all accessories, spare parts or replacement parts, please contact our partsdivision at 1-800-865-6330.
Residents of Canada: Please contact our toll free hotline at 1-888-335-6644 or log onto our website at
www.delonghi.com.
Residents of Mexico:  Please refer to the Limited Warranty statement for Mexico.Please refer to the back page of the manual for De’Longhi addresses.The above warranty is in lieu of all other express warranties and representations. All implied warranties
are limited to the applicable warranty period set forth above. This limitation does not apply if you enterinto an extended warranty with De’Longhi. Some states do not allow limitations on how long an impliedwarranty lasts, so the above exclusions may not apply to you. De’Longhi does not authorize any otherperson or company to assume for it any liability in connection with the sale or use of its appliance.
How does state law apply?
This warranty gives you specific legal rights, and you may also have other rights, which vary from stateto state.

## 2.7. Augmented generation: Answer the question

Now that we've retrieved a relevant passage from our documents, we can create a generation prompt for the Gemini API to produce a final answer. In this example, we retrieved just one passage. However, when working with large datasets, we'll typically retrieve multiple passages and let the Gemini model decide which ones best answer the question. It's perfectly fine if some passages aren't directly related, as the generation step will filter them out.

In [9]:
query_oneline = query.replace("\n", " ")

# This prompt is where you can specify any guidance on tone, or what topics the model should stick to, or avoid.
prompt = f"""You are a helpful and informative bot that answers questions using text from the reference passage included below. 
Be sure to respond in a complete sentence, being comprehensive, including all relevant background information. 
However, you are talking to a non-technical audience, so be sure to break down complicated concepts and 
strike a friendly and converstional tone. If the passage is irrelevant to the answer, you may ignore it.

QUESTION: {query_oneline}
"""

# Add the retrieved documents to the prompt.
for passage in all_passages:
    passage_oneline = passage.replace("\n", " ")
    prompt += f"PASSAGE: {passage_oneline}\n"

print(prompt)

You are a helpful and informative bot that answers questions using text from the reference passage included below. 
Be sure to respond in a complete sentence, being comprehensive, including all relevant background information. 
However, you are talking to a non-technical audience, so be sure to break down complicated concepts and 
strike a friendly and converstional tone. If the passage is irrelevant to the answer, you may ignore it.

QUESTION: Appliences that repaied outside of factory is included warranty?
PASSAGE: 4IMPORTANT SAFEGUARDS When using electrical appliances, basic safety precautions should always be followed to reduce the risk of fire, electric shock, and/pr injury to persons including the following:1. Read all instructions carefully before using the coffee-mill.2 To protect against risk of fire, electrical shock and injury to persons, do not immerse cord, plug or appliance in water or other liquids. 3. The appliance is not intended for use by persons (including children)

In [10]:
answer = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=prompt)

answer.text

"According to the provided text, the warranty does not cover defects or damage to the appliance resulting from repairs or alterations done outside of the De'Longhi factory or authorized service centers. So, if your appliance was repaired somewhere else, the warranty wouldn't apply."